In [3]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import get_file, to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
import numpy as np
import random
import sys
import io

In [4]:
strategy  = tf.distribute.get_strategy()
with strategy.scope():
    print(f"{tf.config.list_physical_devices('GPU') }")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [5]:
data_file_path = '/kaggle/input/kieu-story-txt/truyen_kieu_data_pre.txt'

def load_data() -> tuple:
    rs: map = {}
    ls = []
    tmp_line = ""
    with open(file=data_file_path, mode="rt") as i_f:
        count = 0
        for line in i_f:
            line = line.strip()
            if "" == line: continue
            if count % 2 == 0 and tmp_line!="":
                ls.append(tmp_line.strip())
                tmp_line = ""
                pass
#             tmp_line += (" | " if tmp_line!="" else "") + line
            tmp_line += " "+line
            a_word = line.split()
            for word in a_word:
                if word not in rs:
                    rs[word] = 0
                    pass
                rs[word]+=1
                pass
            pass
            count+=1
        pass
    return rs, ls

In [7]:
bag_of_words, coupled_lines = load_data()
# bag_of_words["|"] = len(coupled_lines)
# bag_of_words["\\"] = len(coupled_lines) - 1
# ix_to_word = [*bag_of_words.keys()]
keys = [*bag_of_words.keys()]
word_to_ix = { w:i for i,w in enumerate(keys) }
ix_to_word = { i:w for i,w in enumerate(keys) }
# coupled_lines
# values = [*bag_of_word.values()]
# sum(values)
# len(bag_of_words)
# ix_to_word
X_train = [[word_to_ix[word] for word in coupled_line.split()] for coupled_line in coupled_lines]

delta_line = 4
# y_train = [line[5] for line in X_train[delta_line:]]
y_train = [*X_train[delta_line:]]
y_train = [e[5] for e in y_train]
for c in range(delta_line):
    y_train.append(X_train[c][5])
    pass

total_word = len(bag_of_words)
max_input_len = 14
X_train = np.asarray(a=X_train, dtype=int)
y_train = np.asarray(y_train) #.reshape(X_train.shape[0], 1)
# print(y_train.shape)
y_train = to_categorical(y_train, num_classes=total_word, dtype=int)
# X_train.shape
# y_train.shape
# len(coupled_lines[0].split())
# coupled_lines

['trăm năm trong cõi người ta chữ tài chữ mệnh khéo là ghét nhau',
 'trải qua một cuộc bể dâu những điều trông thấy mà đau đớn lòng',
 'lạ gì bỉ sắc tư phong trời xanh quen thói má hồng đánh ghen',
 'cảo thơm lần giở trước đèn phong tình có lục còn truyền sử xanh',
 'rằng năm gia tĩnh triều minh bốn phương phẳng lặng hai kinh vững vàng',
 'có nhà viên ngoại họ vương gia tư nghĩ cũng thường thường bực trung',
 'một trai con thứ rốt lòng vương quan là chữ nối dòng nho gia',
 'đầu lòng hai ả tố nga thúy kiều là chị em là thúy vân',
 'mai cốt cách tuyết tinh thần một người một vẻ mười phân vẹn mười',
 'vân xem trang trọng khác vời khuôn trăng đầy đặn nét ngài nở nang',
 'hoa cười ngọc thốt đoan trang mây thua nước tóc tuyết nhường màu da',
 'kiều càng sắc sảo mặn mà so bề tài sắc lại là phần hơn',
 'làn thu thủy nét xuân sơn hoa ghen thua thắm liễu hờn kém xanh',
 'một hai nghiêng nước nghiêng thành sắc đành đòi một tài đành họa hai',
 'thông minh vốn sẵn tư trời pha nghề thi họa đủ mùi ca

In [8]:
model: tf.keras.Model = tf.keras.Sequential(name="LSTM-RNN")
# input = tf.keras.layers.Input(shape=(15))
embedding_1 = tf.keras.layers.Embedding(input_dim=total_word, output_dim=1024, input_length=max_input_len)
bidrect_1 = tf.keras.layers.Bidirectional(\
    layer=tf.keras.layers.LSTM(units=160, return_sequences=True, go_backwards=False),\
    backward_layer=tf.keras.layers.LSTM(units=240, return_sequences=True, go_backwards=True))
bidrect_2 = tf.keras.layers.Bidirectional(layer=tf.keras.layers.LSTM(units=256, return_sequences=True))
dropout = tf.keras.layers.Dropout(rate=0.2)
lstm = tf.keras.layers.LSTM(units=128)
output_1 = tf.keras.layers.Dense(units=total_word, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01))
output_2 = tf.keras.layers.Dense(units=total_word, activation="softmax")

# model.add(input)
model.add(embedding_1)
# model.add(bidrect_1)
model.add(bidrect_2)
model.add(dropout)
model.add(lstm)
model.add(output_1)
model.add(output_2)

model.summary()

Model: "LSTM-RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 14, 1024)          2450432   
                                                                 
 bidirectional_1 (Bidirectio  (None, 14, 512)          2623488   
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 14, 512)           0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               328192    
                                                                 
 dense (Dense)               (None, 2393)              308697    
                                                                 
 dense_1 (Dense)             (None, 2393)              5728842   
                                                          

In [9]:
tf.config.set_soft_device_placement(True) 
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# loss_fn = tf.keras.losses.sparse_categorical_crossentropy
loss_fn = tf.keras.losses.categorical_crossentropy
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=loss_fn,
              metrics=["accuracy"])
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#               loss=loss_fn,
#               metrics=["accuracy"])

In [10]:
# strategy  = tf.distribute.get_strategy()
with strategy.scope():
# with tf.device('/cpu:0'):
# with tf.device("/gpu:0"):
    model.fit(x=X_train, y=y_train, epochs=160, batch_size=64, verbose=1)
    pass

Epoch 1/160
26/26 [==============================] - 16s 151ms/step - loss: 8.4190 - accuracy: 0.0154
Epoch 2/160
26/26 [==============================] - 3s 99ms/step - loss: 6.4588 - accuracy: 0.0123
Epoch 3/160
26/26 [==============================] - 1s 57ms/step - loss: 6.1165 - accuracy: 0.0166
Epoch 4/160
26/26 [==============================] - 2s 61ms/step - loss: 6.0029 - accuracy: 0.0160
Epoch 5/160
26/26 [==============================] - 1s 43ms/step - loss: 5.9148 - accuracy: 0.0160
Epoch 6/160
26/26 [==============================] - 1s 29ms/step - loss: 5.8703 - accuracy: 0.0172
Epoch 7/160
26/26 [==============================] - 1s 36ms/step - loss: 5.8322 - accuracy: 0.0141
Epoch 8/160
26/26 [==============================] - 0s 11ms/step - loss: 5.7608 - accuracy: 0.0129
Epoch 9/160
26/26 [==============================] - 1s 23ms/step - loss: 5.7028 - accuracy: 0.0141
Epoch 10/160
26/26 [==============================] - 1s 29ms/step - loss: 5.6019 - accuracy: 0.01

In [18]:
model.save('/kaggle/working/kieu_story')

In [19]:
import os
import subprocess
from IPython.display import FileLink, display
download_file_name = 'kieu_story'
path = f"/kaggle/working/{download_file_name}"
zip_name = f"/kaggle/working/{download_file_name}.zip"
command = f"zip {zip_name} {path} -r"
result = subprocess.run(command, shell=True, capture_output=True, text=True)
if result.returncode != 0:
    print("Unable to run zip command!")
    print(result.stderr)
display(FileLink(f'{download_file_name}.zip'))


/kaggle/working/kieu_story.zip

In [17]:
num_of_word = max_input_len * 2
seed = "đàn bà dễ có mấy tay đời xưa mấy mặt đời này mấy gan"
# seed = ""
seed_sequence = [word_to_ix[word] for word in seed.split()]
rs = ""
for i in range(num_of_word):
    if i % max_input_len == 0 and i != 0:
#         seed_sequence.append(" \\")
        rs += ("\n")
#         continue
    if i % max_input_len == 6:
#         seed_sequence.append(" | ")
        rs += ("\n")
#         continue
    prediction = model.predict(np.array([seed_sequence]), verbose="0")
    predicted_word_idx = np.argmax(prediction)
    predicted_word = ix_to_word[predicted_word_idx]
    
    seed_sequence.append(predicted_word_idx)
    
    seed_sequence = seed_sequence[1:]
    seed += " " + predicted_word
    rs += " " + predicted_word

print(rs)

 kinh tương hài tà dề thang
 ba đèn chiền đèn thơi hàn vần giầy
 liều bài yên mai mành duyên
 thôi trần nồng ràng thưa tiên tiên tiên
